In [1]:
%matplotlib inline

In [2]:
import tifffile
import napari
import skimage
import matplotlib.pyplot as plt
import numpy as np
import scipy
import PIL
from matplotlib.patches import Ellipse
import os
from napari_animation import Animation
import cv2
import matplotlib

In [3]:
def dilate_multiple(original_img, n_dilations):
    # perform multiple dilations in a row
    dilated_img = np.copy(original_img)
    for k in range(n_dilations):
        dilated_img = skimage.morphology.binary_dilation(dilated_img)
    return dilated_img

In [4]:
def minimize_stack_func(sequence_element, param_array):
    frame = sequence_element
    
    return_list = []
    
    (a, b, alpha, x_0, y_0) = param_array
    
    res = scipy.optimize.minimize(minimize_frame_func, 
        x0=np.asarray((a_0,b_0,alpha_0,x_0,y_0)), 
        args={"frame":frame, "norm_ord":0}, 
        method='Nelder-Mead',
        options={"maxiter":1000})
    
    return_list.append(res['fun'])
    
    return_list.append(res['x'][0])
    return_list.append(res['x'][1])
    
    return_list.append(res['x'][2])
    
    return_list.append(res['x'][3])
    return_list.append(res['x'][4])
    
    return_list.append(res['success'])
    
    return return_list

In [ ]:
def minimize_frame_func(param_array, args_dict, r_min=1, r_max=3):
    # takes a (heavily thresholded) frame and a regularization order as args,
    # ellipse parameters as parameters and computes a negative score
    # the lower negative_score, the better the ellipse fits the frame
    (a, b, alpha, x_0, y_0) = param_array
    a = int(a)
    b = int(b)
    x_0 = int(x_0)
    y_0 = int(y_0)

    frame = args_dict['frame']
    norm_ord = args_dict['norm_ord']

    if (a <= 10) or (b <= 10) or (x_0 <= 10) or (y_0 <= 10):
        negative_score = 0

    elif (a/b<r_min) or (a/b>r_max):
        negative_score = 0
    
    else:
        #negative_score = -1*inner_loop(frame, a, b, alpha, center=(x_0,y_0), norm_ord=norm_ord)
        negative_score = -1*inner_loop(frame, a, b, alpha)
    
    return negative_score

In [ ]:
def inner_loop(frame, a, b, alpha, center=(80,90), norm_ord=2):
    # creates the ellipse in question and computes a positive score,
    # which for norm_ord=2 should be a correlation between the frame and ellipse img
    if len(frame.shape) < 2:
        score = 0
    else:
        ellipse_img = np.zeros_like(frame)
        rr, cc = skimage.draw.ellipse(center[0], center[1], a, b, rotation=np.pi*alpha/180,
            shape=ellipse_img.shape)
        ellipse_img[rr, cc] = 1

        score = np.linalg.norm(np.multiply(ellipse_img.ravel(), frame.ravel()), ord=norm_ord) / np.sqrt(
                np.linalg.norm(ellipse_img.ravel(), ord=norm_ord) * np.linalg.norm(frame.ravel(), ord=norm_ord))

    return score

In [ ]:
# import whole video

video_path = "/data.nst/loidolt/packer_data/pupil_imaging/"
video_name = "2021-03-10_RL123_pupil_2021-03-10-185442_cropped"
video_extension = ".avi"

if video_extension == '.tif':
    tif_path = os.path.join(video_path, video_name+video_extension)
    video_stack = tifffile.imread(tif_path)

elif video_extension == '.avi':
    video_path = os.path.join(video_path, video_name+video_extension)
    cap = cv2.VideoCapture(video_path)
    ret = True

    video_frames = []
    while ret:
        ret, video_img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
        if ret:
            video_frames.append(video_img)
    video_stack = np.stack(video_frames, axis=0)[:,:,:,0] # dimensions (T, H, W, C) - drop last two of C because this

else:
    pass

num_frames = video_stack.shape[0]

In [ ]:
# match histograms to make grey levels comparable across different frames, 
# reference frame should be a "dark" one where it's very hard to make out the pupil

#use this for 2021-03-10_RL117
#reference_frame = restored_video[4824]

#use this for 2021-03-10_RL123
reference_frame = video_stack[9084]


##use this for 2021-03-09_RL117
#reference_frame = video_stack[9078]


rescaled_stack = np.zeros_like(video_stack)
for i_frame in range(num_frames):
    rescaled_stack[i_frame] = skimage.exposure.match_histograms(video_stack[i_frame], reference_frame)

In [ ]:
# rescale image so that only the "pupil-dark" frmaes are left (they are sparse)

## 2021-03-10_RL123
in_range = (0,10)

new_substack = skimage.exposure.rescale_intensity(rescaled_stack, 
                                                  in_range=in_range, out_range=(0,255))

new_substack[new_substack==255] = 0

In [ ]:
# choose initial conditions for ellipse
## 2021-03-10_RL123
x_0, y_0 = 80, 90
a_0, b_0 = 60, 40
alpha_0 = 45

# ellipse elipse-parameter arrays
f_array = np.zeros(num_frames)

axes_array = np.zeros((num_frames,2))
center_array = np.zeros((num_frames,2))
alpha_array = np.zeros(num_frames)

binary_array = np.zeros(num_frames, dtype=bool)

for i_frame in range(100):
    # compute optimal ellipse
    res = scipy.optimize.minimize(minimize_frame_func, 
        x0=np.asarray((a_0,b_0,alpha_0,x_0,y_0)), 
        args={"frame":new_substack[i_frame], "norm_ord":0}, 
        method='Nelder-Mead',
        options={"maxiter":1000})
    
    f_array[i_frame] = res['fun']
    
    axes_array[i_frame][0] = res['x'][0]
    axes_array[i_frame][1] = res['x'][1]
    
    alpha_array[i_frame] = res['x'][2]
    
    center_array[i_frame][0] = res['x'][3]
    center_array[i_frame][1] = res['x'][4]
    
    binary_array[i_frame] = res['success']

In [ ]:
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
x_0, y_0 = 80, 90
a_0, b_0 = 60, 40
alpha_0 = 45

param_array = (a_0, b_0, alpha_0, x_0, y_0)

res_futures = client.map(minimize_stack_func, new_substack, param_array=param_array)

In [ ]:
res_list = client.gather(res_futures)

In [ ]:
res_array = np.asarray(res_list)

f_array = res_array[:,0]
a_array = res_array[:,1]
b_array = res_array[:,2]
alpha_array = res_array[:,3]
x0_array = res_array[:,4]
y0_array = res_array[:,5]
success_array = res_array[:,6]

In [ ]:
plt.plot(res_array[:,1])

In [ ]:
# draw ellipses
ellipse_stack = np.zeros_like(new_substack)

for i_frame in range(num_frames):
    annotated_image = np.zeros_like(new_substack[i_frame])

    angle_rad = -1*alpha_array[i_frame] / 180 * np.pi

    rr, cc = skimage.draw.ellipse_perimeter(int(x_0), int(y_0), 
                                            int(a_array[i_frame]), int(b_array[i_frame]), 
                                            orientation=angle_rad,
                                            shape=annotated_image.shape)
    annotated_image[rr, cc] = 1
    dilated_image = skimage.morphology.dilation(annotated_image, skimage.morphology.disk(radius=1))
    ellipse_stack[i_frame] = dilated_image

In [ ]:
# configure path to save annotated videos in
annotation_path = './annotated_videos'

In [ ]:
# create colour video of mouse face
coloured_faces_stack = np.zeros((*rescaled_stack.shape,3))

coloured_faces_stack[:,:,:,0] = rescaled_stack
coloured_faces_stack[:,:,:,1] = rescaled_stack
coloured_faces_stack[:,:,:,2] = rescaled_stack

# annotate it with blue/red ellipse for confident/unconfident
coloured_faces_stack[:,:,:,0][np.nonzero(ellipse_stack)] = 0
coloured_faces_stack[:,:,:,1][np.nonzero(ellipse_stack)] = 0
coloured_faces_stack[:,:,:,2][np.nonzero(confident_ellipse_stack)] = 255*confident_ellipse_stack[np.nonzero(confident_ellipse_stack)].ravel()

coloured_faces_stack[:,:,:,0][np.nonzero(unconfident_ellipse_stack)] = 255*unconfident_ellipse_stack[np.nonzero(unconfident_ellipse_stack)].ravel()
coloured_faces_stack[:,:,:,1][np.nonzero(unconfident_ellipse_stack)] = 0
coloured_faces_stack[:,:,:,2][np.nonzero(unconfident_ellipse_stack)] = 0

# get video shape
video_shape = (coloured_faces_stack.shape[2], coloured_faces_stack.shape[1])

# configure video path
annotated_video_path = os.path.join(annotation_path, video_name+'MinimallyDaskAnnotated0-10_FixedCenter_allFrames.mp4')

# write video
video_writer = cv2.VideoWriter(annotated_video_path, 
    cv2.VideoWriter_fourcc(*'mp4v'), 30, video_shape, True)
for i_frame in range(num_frames):
    video_frame = cv2.cvtColor(coloured_faces_stack[i_frame].astype(np.uint8), cv2.COLOR_RGB2BGR)
    video_writer.write(video_frame)
video_writer.release()

In [ ]:
array_path = os.path.join(annotation_path, 
                          video_name+'MinimallyDaskAnnotated0-10_FixedCenter_allFrames_resArray.npy')

np.save(array_path, res_array)